In [249]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [250]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df.fillna(0)

df.sample(20)



,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
276809,0xc564ee9f21ed8a2d8e7e76c085740d5e4c5fafbe,in,0xfc41eb1471e4bc3590c499bda639d1917868b33d,0,transfer,2.574062e+04,7,7,Ethereum
199266,0x8691b80192d37520ef1309bd58d7bc1509b58bb5,in,0x9696f59e4d72e237be84ffd425dcad154bf96976,0,transfer,4.700650e+04,1,1,Ethereum
225428,0x6315b4bc7fe2310e4d344ecddc51d5ed6ad4f3c6,in,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0,transfer,1.131922e+05,2,2,Ethereum
275174,0x30cb0d10960d3b826201e23002e936ceaea9896f,out,uniswap_v2,Pair,multiplexBatchSellTokenForToken,4.635434e+04,1,1,Ethereum
84380,0xf89d7b9c864f589bbf53a82105107622b35eaa40,out,0xe9075af7d2f07ebde65e2f865d98000de272ba92,0,transfer,2.653894e+04,4,4,Ethereum
228934,0xa26e73c8e9507d50bf808b7a2ca9d5de4fcc4a04,in,0x0adee49aa72a5a0d2e9d31df047b03d03afc7657,0,0x,1.966544e+04,1,1,Ethereum
113591,0x3442fab38905a20d9180952ca434c71734673bc8,in,0x66c57bf505a85a74609d2c83e94aabb26d691e1f,0,transfer,1.005409e+06,1,1,Ethereum
87225,0x1d54ae3c893de37b90487d13805dedc73cc03a6e,in,OTHER,OTHER,OTHER,1.733789e+02,5,5,Ethereum
34836,0xd954a7049cd136789fdb19304bcd9a2d6998f7c5,in,0xa77996fc97526c9c85b81538bb1822cc22ec8ee2,0,claim,1.961675e+06,1,1,Ethereum
224592,0x3a7a1c811d3bb561181fa771006832fadcd01019,out,0x565e115fbbc0e00d5f6c02d3dcb7010f7e264c6e,0,0x,3.191700e+04,1,1,Ethereum


In [251]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [252]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/4210915753.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [253]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net'\
                # ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share'
                ,'address' \
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [254]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [255]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})

df_action_by_app = df_addr.groupby(['chain','project','address']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
})

df_action_by_app.reset_index(inplace=True)
df_action_by_app = df_action_by_app.groupby(['chain','project']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})


df_action.reset_index(inplace=True)
df_action_by_app.reset_index(inplace=True)
df_action.name = 'df_action'
df_action_by_app.name = 'df_action_by_app'

display(df_action)

ValueError: cannot insert address, already exists

In [ ]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

# df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

In [ ]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

        print(sumdf[['usd_value_out_pct_share_100']].sum())

Dollar-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
2562,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,9.523727,9.255335
2561,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,5.702290,4.862586
1927,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,5.504098,0.000000
2563,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,4.944629,5.567669
2015,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,3.998533,0.000000
536,Arbitrum,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3 - 0...,3.286356,1.529717
343,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,3.059880,0.000000
2568,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,2.852370,3.406012
2020,Arbitrum,gmx,gmx - Vault | 0x364e2311,2.278467,0.000000
1891,Arbitrum,OTHER,OTHER - OTHER | OTHER,2.271972,2.345757


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
663,Optimism,Celer,Celer - Bridge | addLiquidity,17.775668,0.000000
944,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,15.668466,15.976778
761,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,5.628581,0.000000
950,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,3.548631,3.762533
736,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,2.776817,4.930665
633,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,2.594831,0.006871
375,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,0xb1feccccaad6429899a9401dcb19f58a19d1afe6 - 0...,2.323820,0.000000
666,Optimism,Celer,Celer - Bridge | send,1.833812,0.000000
741,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,1.822989,0.000000
804,Optimism,Socket,Socket - 0 | outboundTransferTo,1.733445,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
576,Optimism,Uniswap,23.300819,25.730368
533,Optimism,Celer,20.354813,24.457077
553,Optimism,Multichain,6.270284,6.020605
549,Optimism,Kyberswap,5.180304,5.265309
524,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,2.639540,0.006871
526,Optimism,Aave,2.598473,4.172976
577,Optimism,Uniswap V3,2.470609,3.366635
571,Optimism,Sonne Finance,2.432443,2.811540
357,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,2.323820,0.000000
537,Optimism,Dforce,1.920007,1.210225


usd_value_out_pct_share_100    100.0
dtype: float64


In [ ]:
print('Address-Weighted')
print(df_action.columns)
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # display(tdf)
        tdf = tdf[tdf['address'] >=20]
        sumdf = tdf
        print('Sum by Action')
        # sumdf = tdf.groupby(['chain','project','action']).sum()
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        print(sumdf.columns)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project','address']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )

Address-Weighted
Index(['usd_value', 'usd_value_in', 'usd_value_out', 'usd_value_net',
       'address'],
      dtype='object')


KeyError: 'chain'